In [12]:
#La base de datos "wine" contiene muestras de 3 clases de vinos. Cada una de las observaciones tiene 13 atributos (ver http://archive.ics.uci.edu/ml/datasets/Wine)
#Encuentra la combinación de estrategia de pre-procesamiento + parámetro de regularización C del clasificador logístico
#con la que se obtenga el mejor acierto de clasificación. Se deben contemplar al menos 5 parámetros de C (suficientemente separados entre sí) 
#y 3 estrategias de pre-procesamiento. 

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from numpy import genfromtxt #librería para leer archivos de texto
#Primero debes ingresar a la URL del párrafo superior y descargar la base de datos. La encontrarás como wine.data.
#Una vez descargada, guárdala en la misma ruta del notebook


data = genfromtxt('wine.data', delimiter=',') #almacenamos la base de datos en la variable data
#nos muestra el tipo de varible que es data y el tipo de datos que contiene

#debemos tener en cuenta que para esta base de datos la primera columna es la etiqueta y las siguientes 13 son los 
#atributos.


** Ejercicio 1 **

Desarrolla un código que almacene en la variable X la matriz de observaciones (para este caso sería una matriz de 178x13) y
en Y las etiquetas (vector de 178 posiciones). Adicionalmente debes convertir el vector de etiquetas en un vector de números enteros

In [17]:
X = data[:,1:14]
Y = data[:,0]
Y = Y.astype(np.int64)
print(Y)
print(X.shape)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
(178, 13)


** Ejercicio 2 **

1. Divide la matriz de observaciones en 80% para entrenamiento y 20% para validación
2. Genera las 3 estrategias de preprocesamiento y aplícalas correctamente a las matrices X_train y X_test
3. Instancia los 5 clasificadores (1 para cada valor de C)
4. Entrena cada clasificador para cada una de las estrategias de preprocesamiento
5. Valida los clasificadoes con cada uno de los X_test
6. Concluye. ¿Cuál fue la mejor combinación de C y preprocesamiento?

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
print(X_train.shape, X_test.shape)
#------------------------------------------------
# Remoción
def remocion(X_train, X_test):
    media = X_train.mean(axis = 0)
    desviacion = X_train.std(axis = 0)
    X_train_prepro = (X_train - media)/desviacion
    X_test_prepro = (X_test - media)/desviacion
    return X_train_prepro, X_test_prepro


X_train_preproR, X_test_preproR = remocion(X_train, X_test)


# Escalamiento
#------------------------------------------------
def escalamiento(X_train, X_test):
    maximo = X_train.max(axis = 0) #Calculamos el máximo del train
    minimo = X_train.min(axis = 0) #Calculamos el mínimo del train
    X_train_prepro = (X_train - minimo)/(maximo - minimo) # aplicamos la fórmula
    X_test_prepro = (X_test - minimo)/(maximo - minimo) #Sobre ambos conjuntos
    return X_train_prepro, X_test_prepro #devolvemos los conjuntos procesados

X_train_preproE, X_test_preproE = escalamiento(X_train, X_test)



#Norma1
#------------------------------------------------
def norma1(X_train, X_test): #definición de la función
    norma = np.linalg.norm(X_train, ord = 1, axis = 1) #cálculo de la
    #norma. El parámetro ord define el tipo de norma y el axis en 1
    #indica que se calcula sobre las filas
    #almacenamos el tamaño de la matriz de entrenamiento
    t = X_train.shape #nos arroja una dupla con filas y columnas
    filas = t[0] #tomamos el elemento que corresponde a las filas
    columnas = t[1] #tomamos el elemento que corresponde a las columnas
    #-------------------------------------------------
    norma = np.repeat(norma, columnas) #repite los elemento
    norma = np.reshape(norma,t) #nos da el tamaño de la matriz de entrada
    X_train_prepro = X_train/norma
    
    #------------------------------------------------
    return X_train_prepro,norma

X_train_preproN, X_test_preproN = escalamiento(X_train, X_test)



#-------------------remoción-----------------------------------------
Cc = 0.1
Num_C = 1
listaRemocion = []
for x in range(5):
    
    clasificador = LogisticRegression(C = Cc, random_state = 0)
    clasificador.fit(X_train_preproR,y_train)
    y_predR = clasificador.predict(X_test_preproR)
    error = np.mean(y_test != y_predR)*100
    acierto = np.mean(y_test == y_predR)*100
    cla_1 = ['Clasificador' + str(Num_C),error, acierto]
    listaRemocion.append(cla_1)
    Cc = Cc*10
    
    Num_C = Num_C + 1
    
print(listaRemocion)



#-------------------escalamiento-----------------------------------------

Cc = 0.1
Num_C = 1
listaEscalamiento = []
for x in range(5):
    
    clasificador = LogisticRegression(C = Cc, random_state = 0)
    clasificador.fit(X_train_preproE,y_train)
    y_predE = clasificador.predict(X_test_preproE)
    error = np.mean(y_test != y_predE)*100
    acierto = np.mean(y_test == y_predE)*100
    cla_1 = ['Clasificador' + str(Num_C),error, acierto]
    listaEscalamiento.append(cla_1)
    Cc = Cc*10
    
    Num_C = Num_C + 1
    
print(listaEscalamiento)

#-------------------Norma-----------------------------------------

Cc = 0.1
Num_C = 1
listaNorma = []
for x in range(5):
    
    clasificador = LogisticRegression(C = Cc, random_state = 0)
    clasificador.fit(X_train_preproN,y_train)
    y_predN = clasificador.predict(X_test_preproN)
    error = np.mean(y_test != y_predN)*100
    acierto = np.mean(y_test == y_predN)*100
    cla_1 = ['Clasificador' + str(Num_C),error, acierto]
    listaNorma.append(cla_1)
    Cc = Cc*10
    
    Num_C = Num_C + 1
    
print(listaNorma)

(142, 13) (36, 13)
[['Clasificador1', 0.0, 100.0], ['Clasificador2', 0.0, 100.0], ['Clasificador3', 0.0, 100.0], ['Clasificador4', 0.0, 100.0], ['Clasificador5', 0.0, 100.0]]
[['Clasificador1', 8.333333333333332, 91.66666666666666], ['Clasificador2', 2.7777777777777777, 97.22222222222221], ['Clasificador3', 0.0, 100.0], ['Clasificador4', 0.0, 100.0], ['Clasificador5', 0.0, 100.0]]
[['Clasificador1', 8.333333333333332, 91.66666666666666], ['Clasificador2', 2.7777777777777777, 97.22222222222221], ['Clasificador3', 0.0, 100.0], ['Clasificador4', 0.0, 100.0], ['Clasificador5', 0.0, 100.0]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default so